In [94]:
import pandas as pd
train_data_path = "data/train_AUpWtIz/train.csv"
campaign_data_path = "data/train_AUpWtIz/campaign_data.csv"
coupon_item_mapping_data_path = "data/train_AUpWtIz/coupon_item_mapping.csv"
customer_demographics_data_path = "data/train_AUpWtIz/customer_demographics.csv"
customer_transaction_data_path = "data/train_AUpWtIz/customer_transaction_data.csv"
item_data_path = "data/train_AUpWtIz/item_data.csv"

train_df = pd.read_csv(train_data_path)
campaign_df = pd.read_csv(campaign_data_path)
coupon_item_df = pd.read_csv(coupon_item_mapping_data_path)
customer_demo_df = pd.read_csv(customer_demographics_data_path)
customer_trans_df = pd.read_csv(customer_transaction_data_path)
item_df =  pd.read_csv(item_data_path)

In [95]:
train_df.describe()

,id,campaign_id,coupon_id,customer_id,redemption_status
count,78369.000000,78369.000000,78369.000000,78369.000000,78369.000000
mean,64347.975449,13.974441,566.363243,787.451888,0.009302
std,37126.440855,8.019215,329.966054,456.811339,0.095999
min,1.000000,1.000000,1.000000,1.000000,0.000000
25%,32260.000000,8.000000,280.000000,399.000000,0.000000
50%,64318.000000,13.000000,597.000000,781.000000,0.000000
75%,96577.000000,13.000000,857.000000,1190.000000,0.000000
max,128595.000000,30.000000,1115.000000,1582.000000,1.000000


In [96]:
print(train_df.isnull().sum())
print("-------------------------------")
print(campaign_df.isnull().sum())
print("-------------------------------")
print(coupon_item_df.isnull().sum())
print("-------------------------------")
print(customer_demo_df.isnull().sum())
print("-------------------------------")
print(customer_trans_df.isnull().sum())
print("-------------------------------")
print(item_df.isnull().sum())
print("-------------------------------")

id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
dtype: int64
-------------------------------
campaign_id      0
campaign_type    0
start_date       0
end_date         0
dtype: int64
-------------------------------
coupon_id    0
item_id      0
dtype: int64
-------------------------------
customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64
-------------------------------
date               0
customer_id        0
item_id            0
quantity           0
selling_price      0
other_discount     0
coupon_discount    0
dtype: int64
-------------------------------
item_id       0
brand         0
brand_type    0
category      0
dtype: int64
-------------------------------


In [97]:
customer_demo_df = customer_demo_df[['customer_id', 'age_range', 'rented', 'family_size','income_bracket']]

In [98]:
#train_df has all the numeric columns
import datetime
campaign_df['start_date'] = pd.to_datetime(campaign_df['start_date'])
campaign_df['end_date'] =   pd.to_datetime(campaign_df['end_date'])

campaign_df['month'] = campaign_df['start_date'].dt.month

In [99]:
#check the campaign dataframe
campaign_df.select_dtypes(exclude = ["number"])
#it gives three col "campaign_type" ,"start_date","end_date" are not numbers.
campaign_df_1 = campaign_df.copy()
#USING GETDUMMIES FUNCTION
campaign_df_1 = pd.get_dummies(campaign_df_1,columns = ['campaign_type'])
#df_with_dummies = pd.get_dummies( df, columns = cols_to_transform )

#check the duration and month from the dates.
# campaign_df_1['total_days'] = campaign_df_1['end_date'] - campaign_df_1['start_date']
# campaign_df_1
campaign_df_1

,campaign_id,start_date,end_date,month,campaign_type_X,campaign_type_Y
0,24,2013-10-21,2013-12-20,10,0,1
1,25,2013-10-21,2013-11-22,10,0,1
2,20,2013-07-09,2013-11-16,7,0,1
3,23,2013-08-10,2013-11-15,8,0,1
4,21,2013-09-16,2013-10-18,9,0,1
5,22,2013-09-16,2013-10-18,9,1,0
6,18,2013-10-08,2013-04-10,10,1,0
7,19,2013-08-26,2013-09-27,8,0,1
8,17,2013-07-29,2013-08-30,7,0,1
9,16,2013-07-15,2013-08-16,7,0,1
